In [1]:
import pymongo
import sqlite3
import pandas as pd
import os

In [2]:
# Mongo DB connection
client = pymongo.MongoClient(os.getenv('MONGO_URL'))
db = client['rpg']

In [3]:
# Sqlite Connection
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()

In [8]:
# Function to take in sql queries and return pandas df
def sql_to_df(query, connection=conn):
    return pd.read_sql(query, connection)

In [9]:
# Test function 
sql_to_df('SELECT * FROM armory_weapon LIMIT 5;')

,item_ptr_id,power
0,138,0
1,139,0
2,140,0
3,141,0
4,142,0


In [65]:
# Create Collection for character inventory
inventory = db['inventories']

In [66]:
inv = sql_to_df('SELECT * FROM charactercreator_character_inventory;').to_dict(orient='records')
inventory.insert_many(inv)

In [74]:
# Check results
sql_to_df('SELECT * FROM charactercreator_character_inventory WHERE id=42;')

,id,character_id,item_id
0,42,13,125


In [75]:
inventory.find_one({'id':42})

{'_id': ObjectId('5dc4cbd269839188383464e9'),
 'id': 42,
 'character_id': 13,
 'item_id': 125}

# Let's Create collections for the rest of the tables

In [76]:
table_list = [
    'armory_item', 
    'armory_weapon',
    'charactercreator_character',
    'charactercreator_cleric',
    'charactercreator_fighter',
    'charactercreator_mage',
    'charactercreator_thief',
    'charactercreator_necromancer'
] 

In [77]:
for i in table_list:
    name = i.split('_')[-1] + 's'
    collection = db[name]
    collection.insert_many(sql_to_df(f'SELECT * FROM {i};').to_dict(orient='records'))

# Let's check our results

In [81]:
sql_to_df('SELECT * FROM charactercreator_character WHERE character_id=42;')

,character_id,name,level,exp,hp,strength,intelligence,dexterity,wisdom
0,42,Animi labo,0,0,10,1,1,1,1


In [83]:
db['characters'].find_one({'character_id':42})

{'_id': ObjectId('5dc4ccbb698391883834693e'),
 'character_id': 42,
 'name': 'Animi labo',
 'level': 0,
 'exp': 0,
 'hp': 10,
 'strength': 1,
 'intelligence': 1,
 'dexterity': 1,
 'wisdom': 1}

In [84]:
sql_to_df('SELECT * FROM armory_item WHERE item_id=155;')

,item_id,name,value,weight
0,155,Voluptatum doloremque to,0,0


In [88]:
db['items'].find_one({'item_id':155})

{'_id': ObjectId('5dc4ccbb69839188383468dc'),
 'item_id': 155,
 'name': 'Voluptatum doloremque to',
 'value': 0,
 'weight': 0}

# RPG QUERIES WITH MONGO

## How many total characters?

In [103]:
db['characters'].count_documents({})

302

## How many of each specific subclass

In [109]:
# This is silly, I should've just combined everything
classes = ['clerics', 'fighters', 'mages', 'necromancers', 'thiefs']
amounts = []
for i in classes:
    print(f'{i}: {db[i].count_documents({})}')

clerics: 75
fighters: 68
mages: 108
necromancers: 11
thiefs: 51


# How many total Items?

In [110]:
db['items'].count_documents({})

174

# How many of the items are weapons? How many are not?

## I think it's time to just combine them all into a new collection that has all the information!

In [114]:
# Thanks RICHMOND
char_record = db['chars_full']
char_record.create_index([('character_id', pymongo.ASCENDING)], unique=True)
characters = sql_to_df('SELECT * FROM charactercreator_character').to_dict(orient='records')
try:
    char_record.insert_many(characters)
except pymongo.errors.BulkWriteError as e:
        print('Record already written')

In [116]:
# Add classes
base_classes = [
    'charactercreator_cleric',
    'charactercreator_fighter',
    'charactercreator_mage',
    'charactercreator_thief',
    'charactercreator_necromancer'
]

char_classes = {}

operations = []
for cls in base_classes:
    for rec in sql_to_df(f'SELECT * FROM {cls}').to_dict(orient='records'):
        char_foreign_key = 'mage_ptr_id' if cls == 'charactercreator_necromancer'\
        else 'character_ptr_id'
        character_id = rec.pop(char_foreign_key)
        class_name = cls.split('_')[-1]
        rec[class_name] = True
        char_classes.setdefault(character_id,[]).append(class_name)
        rec['classes'] = char_classes[character_id]
        rec['items'] = []
        query = {'character_id': character_id}
        operations.append(pymongo.UpdateOne(query, {'$set': rec}, upsert=True))
char_record.bulk_write(operations)

In [117]:
# Add items
operations = []
from math import isnan
for rec in sql_to_df('''
        SELECT character_id, ai.item_id, name, value, weight, power, weapon
        FROM charactercreator_character_inventory AS ci 
        JOIN armory_item AS ai ON ci.item_id=ai.item_id
        LEFT JOIN (SELECT *, True as weapon FROM armory_weapon) 
        AS aw on aw.item_ptr_id = ai.item_id
        ORDER BY ai.item_id
    ''').to_dict(orient='records'):
    character_id = rec.pop('character_id')
    keys = list(rec.keys())
    for key in keys:
        v = rec[key]     
        if type(v) == float and isnan(v):
            rec.pop(key)
    if rec.get('weapon') == 1:
        rec['weapon'] = True
    query = {'character_id': character_id}
    operations.append(pymongo.UpdateOne(query, {'$addToSet': {'items': rec}}, upsert=True))
char_record.bulk_write(operations)

##### ^This is dope

# Back to the questions! How many of the items are weapons? How many are not?

In [148]:
weps = char_record.aggregate([
    {'$unwind': '$items'},
    {'$match': {'items.weapon': True}},
    {'$count': 'Total Weapons'}
])

In [149]:
list(weps)

[{'Total Weapons': 203}]

In [159]:
list(char_record.aggregate([
    {'$unwind': '$items'},
    {'$match': {'items.weapon': {'$ne':True}}},
    {'$count': 'Non-Weapons'}
]))

[{'Non-Weapons': 695}]

# How many items does each character have? (Return first 20 rows)

In [161]:
list(char_record.aggregate([
    {'$project': {'_id': '$character_id', 'name': '$name', 'item_count': {'$size': '$items'}}},
    { '$limit': 20 }
]))

[{'_id': 1, 'name': 'Aliquid iste optio reiciendi', 'item_count': 3},
 {'_id': 2, 'name': 'Optio dolorem ex a', 'item_count': 3},
 {'_id': 3, 'name': 'Minus c', 'item_count': 2},
 {'_id': 4, 'name': 'Sit ut repr', 'item_count': 4},
 {'_id': 5, 'name': 'At id recusandae expl', 'item_count': 4},
 {'_id': 6, 'name': 'Non nobis et of', 'item_count': 1},
 {'_id': 7, 'name': 'Perferendis', 'item_count': 5},
 {'_id': 8, 'name': 'Accusantium amet quidem eve', 'item_count': 3},
 {'_id': 9, 'name': 'Sed nostrum inventore error m', 'item_count': 4},
 {'_id': 10, 'name': 'Harum repellendus omnis od', 'item_count': 4},
 {'_id': 11, 'name': 'Itaque ut commodi,', 'item_count': 3},
 {'_id': 12, 'name': 'Molestiae quis', 'item_count': 3},
 {'_id': 13, 'name': 'Ali', 'item_count': 4},
 {'_id': 14, 'name': 'Tempora quod optio possimus il', 'item_count': 4},
 {'_id': 15, 'name': 'Sed itaque beatae pari', 'item_count': 4},
 {'_id': 16, 'name': 'Quam dolor', 'item_count': 1},
 {'_id': 17, 'name': 'Molestias

# How many weapons does each character have? (Return first 20)

In [187]:
list(char_record.aggregate([
    {'$unwind': '$items'},
    {'$project': {'character_id':1, 'name':'$name', 'weaps': {'$cond' : [{'$eq':['$items.weapon', True]}, 1, 0]}}},
    {'$group': {'_id':'$character_id', 'name':{'$first':'$name'}, 'Number of Weapons': {'$sum':'$weps'}}},
    {'$sort' : {'_id':1}},
    {'$limit': 20}
]))

[{'_id': 1, 'name': 'Aliquid iste optio reiciendi', 'Number of Weapons': 0},
 {'_id': 2, 'name': 'Optio dolorem ex a', 'Number of Weapons': 0},
 {'_id': 3, 'name': 'Minus c', 'Number of Weapons': 0},
 {'_id': 4, 'name': 'Sit ut repr', 'Number of Weapons': 0},
 {'_id': 5, 'name': 'At id recusandae expl', 'Number of Weapons': 0},
 {'_id': 6, 'name': 'Non nobis et of', 'Number of Weapons': 0},
 {'_id': 7, 'name': 'Perferendis', 'Number of Weapons': 0},
 {'_id': 8, 'name': 'Accusantium amet quidem eve', 'Number of Weapons': 0},
 {'_id': 9, 'name': 'Sed nostrum inventore error m', 'Number of Weapons': 0},
 {'_id': 10, 'name': 'Harum repellendus omnis od', 'Number of Weapons': 0},
 {'_id': 11, 'name': 'Itaque ut commodi,', 'Number of Weapons': 0},
 {'_id': 12, 'name': 'Molestiae quis', 'Number of Weapons': 0},
 {'_id': 13, 'name': 'Ali', 'Number of Weapons': 0},
 {'_id': 14, 'name': 'Tempora quod optio possimus il', 'Number of Weapons': 0},
 {'_id': 15, 'name': 'Sed itaque beatae pari', 'Num

# On average, how many items does each Character have?

In [193]:
list(char_record.aggregate([
    {'$unwind': '$items'},
    {'$group': {'_id': '$character_id','count':{'$sum':1}}},
    {'$group': {'_id': '', 'Average Items':{'$avg':'$count'}}},
]))

[{'_id': '', 'Average Items': 2.9735099337748343}]

# On average, how many weapons does each Character have?

In [195]:
list(char_record.aggregate([
    {'$unwind': '$items'},
    {'$project': {'character_id': 1, 'weapons': 
                  {'$cond': [ { '$eq': ["$items.weapon", True] }, 1, 0]}}},
    {'$group': {'_id': '$character_id', 'weapons': {'$sum':'$weapons'} }},
    {'$group': {'_id':'', 'Average weapons':{'$avg':'$weapons'}}},
    
]))

[{'_id': '', 'Average weapons': 0.6721854304635762}]

I just have to say that there is no way I would have been able to do this without your help Richmond. It would have taken me a full week just to look up all of the pymongo commands, let alone what they look like in a mongo query.